In [42]:
import pandas as pd
from carps.analysis.gather_data import normalize_logs, get_interpolated_performance_df
from numpy import trapz
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px 
import itertools
import pandas as pd
from carps.analysis.generate_report import *
from carps.analysis.performance_over_time import plot_performance_over_time


# Old

In [3]:
all_experiments = []
all_trajectories = []
current_max_id = 0

for subdir in os.listdir('../experimenter_loose/'):
    experiment_config = pd.read_parquet(f"../experimenter_loose/{subdir}/experiment_config.parquet")
    trials = pd.read_parquet(f"../experimenter_loose/{subdir}/trials.parquet")

    experiment_config = experiment_config[~experiment_config['optimizer_id'].str.startswith("hypershap")]
    experiment_config = experiment_config[~experiment_config['optimizer_id'].str.startswith("random")]
    valid_ids = experiment_config['ID'].unique()
    trials = trials[trials['experiment_id'].isin(valid_ids)]

    id_mapping = {old_id: new_id for old_id, new_id in zip(
        experiment_config['ID'], 
        range(current_max_id, current_max_id + len(experiment_config))
    )}

    experiment_config['ID'] = experiment_config['ID'].map(id_mapping)
    trials['experiment_id'] = trials['experiment_id'].map(id_mapping)

    all_experiments.append(experiment_config)
    all_trajectories.append(trials)

    current_max_id += len(experiment_config)

# Concatenate all
experiment_config_all = pd.concat(all_experiments, ignore_index=True)
trials_all = pd.concat(all_trajectories, ignore_index=True)
trials_all.to_parquet('../experimenter/results/trials.parquet')    
experiment_config_all.to_parquet('../experimenter/results/experiment_config.parquet') 


In [23]:
trials_all = pd.read_parquet('../experimenter/results/trials.parquet')    
experiment_config_all = pd.read_parquet('../experimenter/results/experiment_config.parquet') 


In [6]:
exp_test = experiment_config_all[experiment_config_all['subset_id']=='test']
trials_test = trials_all[trials_all['experiment_id'].isin(exp_test['ID'].unique())]
trials_test.to_parquet('../experimenter/test/trials.parquet')    
exp_test.to_parquet('../experimenter/test/experiment_config.parquet') 

In [7]:
exp_train = experiment_config_all[experiment_config_all['subset_id']=='dev']
trials_train = trials_all[trials_all['experiment_id'].isin(exp_train['ID'].unique())]
trials_train.to_parquet('../experimenter/train/trials.parquet')    
exp_train.to_parquet('../experimenter/train/experiment_config.parquet') 

In [25]:
exp_train = experiment_config_all[experiment_config_all['subset_id']=='dev']
exp_train = exp_train[exp_train['optimizer_id'].isin(['SMAC3-MO-RF', 'fanova_adjust_cs_default_constant_adjust_prev_cfgs_set_to_random_thresh_0_lin_0', 'fanova_adjust_cs_incumbent_cs_proba_hpi_adjust_prev_cfgs_set_to_random_thresh_0_lin_0','fanova_adjust_cs_default_constant_adjust_prev_cfgs_set_to_incumbent_thresh_0_lin_0', 'fanova_adjust_cs_incumbent_cs_proba_hpi_thresh_0.5', 'fanova_adjust_cs_default_constant_adjust_prev_cfgs_set_to_incumbent_thresh_0.75'])]
trials_train = trials_all[trials_all['experiment_id'].isin(exp_train['ID'].unique())]
trials_train.to_parquet('../experimenter/mine/trials.parquet')    
exp_train.to_parquet('../experimenter/mine/experiment_config.parquet') 

# Read data

In [126]:
df = pd.read_parquet('../results/logs_complete.parquet')
df = normalize_logs(df)
if 'set' not in df.columns:
    df["set"] = '0'
df = get_interpolated_performance_df(df, n_points=50)


[16:29:07] INFO     Start normalization...                                                       ]8;id=740770;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=598554;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#626\626]8;;\

           INFO     Normalize n_trials...                                                        ]8;id=827441;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=970319;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#627\627]8;;\

           INFO     Normalize cost...                                                            ]8;id=157286;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=334764;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#629\629]8;;\

[16:29:09] INFO     Calc normalized incumbent cost...                                            ]8;id=197150;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=47693;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#638\638]8;;\

[16:29:14] INFO     Normalize time...                                                            ]8;id=961882;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=456293;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#658\658]8;;\

[16:29:21] INFO     Done.                                                                        ]8;id=373912;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=213926;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#661\661]8;;\

           INFO     Create dataframe for neat plotting by aligning x-axis / interpolating        ]8;id=879779;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=26487;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#735\735]8;;\
                    budget.                                                                                        

In [127]:
df.shape

(3099372, 14)

In [128]:
# # do prepocessing and create subsets: train/test etc.
# df_train = df[df['subset_id']=='dev']
# df_test = df[df['subset_id']=='test']

In [129]:
def expand_optimizer_id(df: pd.DataFrame, optimizer_id_col: str = 'optimizer_id') -> pd.DataFrame:
    def parse_optimizer_id(opt_id: str) -> dict:
        info = {
            'hpi_method': None,
            'adjust_cs': None,
            'cs_method': 'no',
            'adjust_pc': 'False',
            'pc_method': 'False',
            'thresh': None,
            'adjust_method': None
        }
        parts = opt_id.split('_')
        info['hpi_method'] = parts[0]
        
        if 'cs' in parts:
            info['cs_method'] = parts[parts.index('cs')+1]
        if info['cs_method']!='no':
            info['adjust_cs'] = 'True'
            info['adjust_method'] = 'adjust_cs_only'
        if 'cs_proba_hpi' in opt_id:
            info['adjust_cs'] = 'cs_proba_hpi'
        if 'set_to' in opt_id:
            info['adjust_pc'] = ' '.join(parts[parts.index('cfgs')+1:parts.index('cfgs')+3])
            info['pc_method'] = parts[parts.index('to')+1]
            if info['adjust_method'] == 'adjust_cs_only':
                info['adjust_method'] = 'both'
            else:
                info['adjust_method'] = 'adjust_pc_only'
        if 'thresh' in parts:
            info['thresh'] = parts[parts.index('thresh')+1]
        
        return info

    # Apply the parser to each row
    parsed = df[optimizer_id_col].apply(parse_optimizer_id)
    parsed_df = pd.DataFrame(parsed.tolist())

    # Merge with the original DataFrame
    return pd.concat([df, parsed_df], axis=1)

def check_if_complete_for_max_trials(in_df):
    max_trials = in_df['n_trials_norm'].max()
    filtered_df = in_df[in_df['n_trials_norm'] == max_trials]
    miss_dict = {}
    seeds = filtered_df['seed'].unique()
    expected_combinations = set(itertools.product(tasks[in_df['benchmark_id'].iloc[0]], [max_trials], seeds))
    incomplete = False
    for optimizer_id, group in filtered_df.groupby('optimizer_id'):
        actual_combinations = set(group[['task_id', 'n_trials_norm', 'seed']].itertuples(index=False, name=None))
        missing = expected_combinations - actual_combinations
        if missing:
            print(f"Optimizer {optimizer_id} is not complete for max n_trials_norm = {max_trials}: {len(missing)} combinations missing.")
            miss_dict[optimizer_id] = missing
            incomplete = True
    if not incomplete:
        print('All data complete!')
        return None
    return miss_dict

def read_data(path):
    df = pd.read_parquet(path)
    df = normalize_logs(df)
    df['n_trials_norm'] = df.groupby('experiment_id')['time'].rank(method='dense', pct=True).round(2)
    if 'set' not in df.columns:
        df["set"] = '0'
    df_random = df[df['optimizer_id'].str.startswith('random_')]
    df = df[~df['optimizer_id'].str.startswith('random_')]
    # check_if_complete_for_max_trials(df)
    return df, df_random

def get_perf(df):
    perf = get_interpolated_performance_df(df, n_points=50)
    # missing = check_if_complete_for_max_trials(df)
    perf = expand_optimizer_id(perf)
    # perf = perf[perf['adjust_method'].notna()]
    return perf

In [130]:
df = expand_optimizer_id(df)

In [131]:
task = 'all'
figure_dir = f"figures_{task}"
# df = df[df['task_id'].str.contains(task)]

# Ablation

In [132]:
def plot_one_thing(thing, df_plot=None):
    if df_plot is None:
        df_plot = df.copy(deep=True)
        df_plot = df_plot[df_plot['optimizer_id']!='SMAC3-MO-RF']
        df_plot['optimizer_id'] = df_plot[thing]
        df_plot = df_plot[df_plot['optimizer_id'].notna()]
        # df_plot = pd.concat([df_plot, base], ignore_index=True)
        plot_performance_per_task(df_plot, output_dir=f"figure_dir/ablation/{thing}", replot=True)
        if len(df_plot['optimizer_id'].unique()) > 2:
            plot_critical_difference(df_plot, output_dir=f"figure_dir/ablation/{thing}", replot=True)
            plot_finalperfbarplot(df_plot, output_dir=f"figure_dir/ablation/{thing}", replot=True)
            plot_ranks_over_time(df_plot, output_dir=f"figure_dir/ablation/{thing}", replot=True)

    # Average over tasks
    avg_task = df_plot.groupby(["optimizer_id", "seed", "n_trials_norm"]).agg(
        mean_perf=("trial_value__cost_inc_norm", "mean")
    ).reset_index()

    # Average over seeds
    avg_seed = avg_task.groupby(["optimizer_id", "n_trials_norm"]).agg(
        mean=("mean_perf", "mean"),
        sem=("mean_perf", "sem")
    ).reset_index()

    auc_per_seed = (
        df_plot.groupby(["optimizer_id", "seed"])
        .apply(lambda group: trapz(group.sort_values("n_trials_norm")["trial_value__cost_inc_norm"],
                                   group.sort_values("n_trials_norm")["n_trials_norm"]))
        .reset_index(name="auc")
    )

    # Compute mean and sem AUC per optimizer
    auc_values = auc_per_seed.groupby("optimizer_id").agg(
        mean=('auc', 'mean'),
        sem=('auc', 'sem')
    ).sort_values("mean", ascending=False).reset_index()


    fig = make_subplots(
        rows=1, cols=2,
        # subplot_titles=("Performance over Time (Normalized)", "AUC per Optimizer"),
        column_widths=[0.6, 0.4],
        horizontal_spacing=0.15
    )
    colors = px.colors.qualitative.Set1
    # Line plot with shaded error (left)
    print(avg_seed["optimizer_id"].unique())
    for i, optimizer in enumerate(avg_seed["optimizer_id"].unique()):
        df_opt = avg_seed[avg_seed["optimizer_id"] == optimizer]
        
        line_color = colors[i % len(colors)]

        # Mean line
        fig.add_trace(go.Scatter(
            x=df_opt["n_trials_norm"],
            y=df_opt["mean"],
            mode="lines",
            name=optimizer,
            line=dict(width=2, color=line_color),
        ), row=1, col=1)

        # Shaded error (sem)
        fig.add_trace(go.Scatter(
            x=pd.concat([df_opt["n_trials_norm"], df_opt["n_trials_norm"][::-1]]),
            y=pd.concat([
                df_opt["mean"] - df_opt["sem"],
                (df_opt["mean"] + df_opt["sem"])[::-1]
            ]),
            fill='toself',
            fillcolor=line_color.replace(')', ',0.1)').replace('rgb', 'rgba'),
            line=dict(color='rgba(255,255,255,0)'),
            hoverinfo="skip",
            showlegend=False
        ), row=1, col=1)

    # Bar chart with error bars (right)
    fig.add_trace(go.Bar(
        x=auc_values["mean"],
        y=auc_values["optimizer_id"],
        orientation="h",
        error_x=dict(type='data', array=auc_values["sem"]),
        marker=dict(color="skyblue", line=dict(color="black", width=1)),
        showlegend=False
    ), row=1, col=2)

    # -------------------------
    # 3. Layout & Font
    # -------------------------

    fig.update_layout(
        height=400,
        width=1800,
        template="plotly_white",
        font=dict(size=16),
        legend=dict(title="Optimizers", font=dict(size=16)),
        margin=dict(t=60, l=50, r=50, b=50)
    )

    fig.update_xaxes(title_text="Normalized Trials", row=1, col=1)
    fig.update_yaxes(title_text="Normalized Inc-Performance", row=1, col=1)
    fig.update_xaxes(title_text="AUC", row=1, col=2)
    fig.update_yaxes(title_text="Optimizer", row=1, col=2)

    fig.show()


In [133]:
# Which HPI method
plot_one_thing('hpi_method')

['fanova']


/tmp/ipykernel_27098/3449863810.py:27: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [134]:
# Which thresholding?
plot_one_thing('thresh')

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:276: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/run_autorank.py:322: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/run_autorank.py:421: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/run_autorank.py:198: UserWarning:

Result is not significant and results of the plot may be misleading!



[16:30:34] INFO     Create dataframe for neat plotting by aligning x-axis / interpolating        ]8;id=675835;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=385454;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#735\735]8;;\
                    budget.                                                                                        

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/anal

['0-075-0' '0-down' '0-down-0' '0-down-0-fine' '0-up-0' '05' '075' 'cos'
 'down-0']


/tmp/ipykernel_27098/3449863810.py:27: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [135]:
plot_one_thing('adjust_cs')

['True' 'cs_proba_hpi']


/tmp/ipykernel_27098/3449863810.py:27: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [136]:
plot_one_thing('cs_method')

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:276: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/run_autorank.py:322: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/run_autorank.py:421: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/run_autorank.py:198: UserWarning:

Result is not significant and results of the plot may be misleading!



[16:31:05] INFO     Create dataframe for neat plotting by aligning x-axis / interpolating        ]8;id=500202;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=418460;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#735\735]8;;\
                    budget.                                                                                        

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/anal

['default' 'incumbent' 'no' 'random']


/tmp/ipykernel_27098/3449863810.py:27: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [137]:
plot_one_thing('adjust_pc')

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/run_autorank.py:322: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/run_autorank.py:421: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown



[16:31:29] INFO     Create dataframe for neat plotting by aligning x-axis / interpolating        ]8;id=92547;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=930325;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#735\735]8;;\
                    budget.                                                                                        

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/anal

['False' 'true no' 'true retrain']


/tmp/ipykernel_27098/3449863810.py:27: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [138]:
plot_one_thing('pc_method')

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/run_autorank.py:322: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/run_autorank.py:421: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown



[16:31:56] INFO     Create dataframe for neat plotting by aligning x-axis / interpolating        ]8;id=157906;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=942714;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#735\735]8;;\
                    budget.                                                                                        

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/anal

['False' 'default' 'incumbent' 'random']


/tmp/ipykernel_27098/3449863810.py:27: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [139]:
plot_one_thing('adjust_method')

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/run_autorank.py:322: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/run_autorank.py:421: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown



[16:32:15] INFO     Create dataframe for neat plotting by aligning x-axis / interpolating        ]8;id=154185;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=29458;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#735\735]8;;\
                    budget.                                                                                        

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/anal

['adjust_cs_only' 'adjust_pc_only' 'both']


/tmp/ipykernel_27098/3449863810.py:27: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



# Create Table

In [140]:
auc_per_seed = (
        df.groupby(["optimizer_id", "seed"])
        .apply(lambda group: trapz(group.sort_values("n_trials_norm")["trial_value__cost_inc_norm"],
                                   group.sort_values("n_trials_norm")["n_trials_norm"]))
        .reset_index(name="auc")
    )

# Compute mean and sem AUC per optimizer
auc_values = auc_per_seed.groupby("optimizer_id").agg(
    auc_mean=('auc', 'mean'),
    auc_sem=('auc', 'sem')
).sort_values("auc_mean", ascending=False).reset_index()


/tmp/ipykernel_27098/1979927433.py:3: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [141]:
merge = pd.merge(auc_values, df, on='optimizer_id', how='left')
merge = merge.drop_duplicates(subset=['optimizer_id'])
merge = merge[['optimizer_id', 'auc_mean', 'auc_sem']]
# merge = merge[['optimizer_id', 'hpi_method', 'adjust_cs', 'cs_method', 'pc_method', 'thresh', 'adjust_method', 'auc_mean', 'auc_sem']]
final_perf = df[df['n_trials_norm'] == 1].groupby(['optimizer_id','seed']).agg({'trial_value__cost_inc_norm':'mean'}).groupby('optimizer_id').agg(
    final_perf_mean=('trial_value__cost_inc_norm', 'mean'),
    final_perf_sem=('trial_value__cost_inc_norm', 'sem')
).sort_values("final_perf_mean", ascending=False).reset_index()

merge = pd.merge(final_perf, merge, on='optimizer_id', how='right')
merge = merge[['optimizer_id', 'auc_mean','auc_sem', 'final_perf_mean', 'final_perf_sem']].sort_values(by='final_perf_mean', ascending=True)
merge = merge.apply(lambda col: col.round(3) if col.name in ['auc_mean', 'auc_sem', 'final_perf_mean', 'final_perf_sem'] else col)
merge.head(20).to_latex()

'\\begin{tabular}{llrrrr}\n\\toprule\n & optimizer_id & auc_mean & auc_sem & final_perf_mean & final_perf_sem \\\\\n\\midrule\n441 & fanova_adjust_cs_default_adjust_prev_cfgs_true_no_retrain_set_to_incumbent_thresh_0-075-0 & 0.178000 & 0.005000 & 0.073000 & 0.002000 \\\\\n418 & fanova_adjust_cs_no_adjust_prev_cfgs_true_no_retrain_set_to_random_thresh_cos & 0.190000 & 0.004000 & 0.078000 & 0.003000 \\\\\n440 & fanova_adjust_cs_default_adjust_prev_cfgs_true_no_retrain_set_to_incumbent_thresh_0-down & 0.180000 & 0.006000 & 0.080000 & 0.007000 \\\\\n433 & fanova_adjust_cs_default_cs_proba_hpi_adjust_prev_cfgs_true_no_retrain_set_to_incumbent_thresh_cos & 0.186000 & 0.012000 & 0.082000 & 0.003000 \\\\\n414 & fanova_adjust_cs_default_cs_proba_hpi_adjust_prev_cfgs_true_no_retrain_set_to_incumbent_thresh_down-0 & 0.190000 & 0.008000 & 0.082000 & 0.003000 \\\\\n431 & fanova_adjust_cs_no_adjust_prev_cfgs_true_no_retrain_set_to_incumbent_thresh_cos & 0.187000 & 0.010000 & 0.082000 & 0.004000 \\\\

In [142]:
merge.sort_values(by='auc_mean', ascending=True)

,optimizer_id,auc_mean,auc_sem,final_perf_mean,final_perf_sem
441,fanova_adjust_cs_default_adjust_prev_cfgs_true...,0.178,0.005,0.073,0.002
440,fanova_adjust_cs_default_adjust_prev_cfgs_true...,0.180,0.006,0.080,0.007
439,fanova_adjust_cs_random_cs_proba_hpi_adjust_pr...,0.182,0.003,0.084,0.004
438,fanova_adjust_cs_no_adjust_prev_cfgs_true_no_r...,0.183,0.006,0.090,0.003
437,fanova_adjust_cs_random_cs_proba_hpi_adjust_pr...,0.183,0.005,0.084,0.003
...,...,...,...,...,...
2,fanova_adjust_cs_default_adjust_prev_cfgs_true...,0.268,0.054,0.213,0.050
4,fanova_adjust_cs_no_adjust_prev_cfgs_true_retr...,0.268,0.054,0.211,0.050
3,fanova_adjust_cs_random_cs_proba_hpi_adjust_pr...,0.268,0.056,0.215,0.049
0,fanova_adjust_cs_random_cs_proba_hpi_adjust_pr...,0.269,0.055,0.213,0.051


# Final Perfromance

In [143]:
# plot best fanova vs best hypershap vs ParEGO
df_smac = df[df['optimizer_id']=='SMAC3-MO-RF']
best_final = merge[merge['final_perf_mean'] == merge['final_perf_mean'].min()]['optimizer_id'].values[0]
fanova_final = df[df['optimizer_id']==best_final]
# fanova_final['optimizer_id'] = 'fANOVA-ParEGO-final'
best_auc = merge[merge['auc_mean'] == merge['auc_mean'].min()]['optimizer_id'].values[0]
fanova_auc = df[df['optimizer_id']==best_auc]
# fanova_auc['optimizer_id'] = 'fANOVA-ParEGO-auc'
# hypershap_final = perf_all[perf_all['optimizer_id']=='hypershap_adjust_cs_random_adjust_prev_cfgs_set_to_random_thresh_0.75']
# hypershap_final['optimizer_id'] = 'HyperSHAP-ParEGO-final'
# hypershap_auc = perf_all[perf_all['optimizer_id']=='hypershap_adjust_cs_default_constant_adjust_prev_cfgs_set_to_incumbent_dynamic_decay_down']
# hypershap_auc['optimizer_id'] = 'HyperSHAP-ParEGO'
df_plot = pd.concat([df_smac, fanova_auc, fanova_final], ignore_index=True)

plot_one_thing('optimizer_id', df_plot=df_plot)

['SMAC3-MO-RF'
 'fanova_adjust_cs_default_adjust_prev_cfgs_true_no_retrain_set_to_incumbent_thresh_0-075-0']


/tmp/ipykernel_27098/3449863810.py:27: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [144]:
top5 = merge[merge['auc_mean'].isin(merge['auc_mean'].sort_values().head(3).values)]['optimizer_id'].values
top5 = pd.concat([df_smac, df[df['optimizer_id'].isin(top5)]])
plot_performance_over_time(top5,  y='trial_value__cost_inc_norm', figure_filename=f'{figure_dir}/perf_over_time')
plot_critical_difference(top5, output_dir=figure_dir, replot=True)
plot_performance_per_task(top5, output_dir=figure_dir, replot=True)
plot_finalperfbarplot(top5, output_dir=figure_dir, replot=True)
plot_ranks_over_time(top5, output_dir=figure_dir, replot=True)


/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:276: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/run_autorank.py:322: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/run_autorank.py:415: UserWarning:

Tight layout not applied. The left and right margins cannot be made large enough to accommodate all Axes decorations.

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/run_autorank.py:421: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/run_autorank.py:198: UserWarning:

Result is not significant and results of the plot may be misleading!



[16:32:36] INFO     Create dataframe for neat plotting by aligning x-axis / interpolating        ]8;id=215964;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=416551;file:///home/daphne/Code/hpi_parego/CARP-S/carps/analysis/gather_data.py#735\735]8;;\
                    budget.                                                                                        

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/analysis/generate_report.py:108: UserWarning:

Result is not significant and results of the plot may be misleading!

/home/daphne/Code/hpi_parego/CARP-S/carps/anal

[{'task_type': 'multi-objective',
  'set': '0',
  'task_id': None,
  'filename': 'figures_all/rank_multi-objective_0',
  'plot_type': 'rank_over_time',
  'plot_type_pretty': 'Rank over Time',
  'explanation': 'The rank of each optimizer over time compares which optimizer performs better, the lower the rank the better. For each optimizer and task, the performance is averaged over seeds to obtain an estimate of the performance. The rank is then calculated per step and task with the same approach as for the critical difference diagram.'}]

# Random

In [186]:
fanova = perf_all[perf_all['optimizer_id']=='fanova_adjust_cs_default_constant_adjust_prev_cfgs_set_to_incumbent_thresh_0.75']
fanova_rnd = rnd_all[rnd_all['optimizer_id']=='random_adjust_cs_default_constant_adjust_prev_cfgs_set_to_incumbent_thresh_0.75']
fanova = pd.merge(fanova, fanova_rnd, on=['task_id', 'seed', 'n_trials_norm'], suffixes=('_final', '_rnd'))
fanova['trial_value__cost_inc_norm_diff'] = fanova['trial_value__cost_inc_norm_rnd'] - fanova['trial_value__cost_inc_norm_final']
fanova['optimizer_id'] = 'fANOVA-ParEGO'


hypershap = perf_all[perf_all['optimizer_id']=='hypershap_adjust_cs_default_constant_adjust_prev_cfgs_set_to_incumbent_dynamic_decay_down']
hypershap_rnd = rnd_all[rnd_all['optimizer_id']=='random_adjust_cs_default_constant_adjust_prev_cfgs_set_to_incumbent_dynamic_decay_down']
hypershap = pd.merge(hypershap, hypershap_rnd, on=['task_id', 'seed', 'n_trials_norm'], suffixes=('_final', '_rnd'))
hypershap['trial_value__cost_inc_norm_diff'] = hypershap['trial_value__cost_inc_norm_rnd'] - hypershap['trial_value__cost_inc_norm_final']
hypershap['optimizer_id'] = 'HyperSHAP-ParEGO'

df_plot = pd.concat([hypershap, fanova], ignore_index=True)

# Average over tasks
avg_task = df_plot.groupby(["optimizer_id", "seed", "n_trials_norm"]).agg(
    mean_perf=("trial_value__cost_inc_norm_diff", "mean")
).reset_index()

# Average over seeds
avg_seed = avg_task.groupby(["optimizer_id", "n_trials_norm"]).agg(
    mean=("mean_perf", "mean"),
    sem=("mean_perf", "sem")
).reset_index()

auc_per_seed = (
    df_plot.groupby(["optimizer_id", "seed"])
    .apply(lambda group: trapz(group.sort_values("n_trials_norm")["trial_value__cost_inc_norm_diff"],
                                group.sort_values("n_trials_norm")["n_trials_norm"]))
    .reset_index(name="auc")
)

colors = px.colors.qualitative.Set1
fig = go.Figure()
for i, optimizer in enumerate(avg_seed["optimizer_id"].unique()):
    df_opt = avg_seed[avg_seed["optimizer_id"] == optimizer]
    line_color = colors[i % len(colors)]

    # Mean line
    fig.add_trace(go.Scatter(
        x=df_opt["n_trials_norm"],
        y=df_opt["mean"],
        mode="lines",
        name=optimizer,
        line=dict(width=2, color=line_color),
    ))

    # Shaded error (sem)
    fig.add_trace(go.Scatter(
        x=pd.concat([df_opt["n_trials_norm"], df_opt["n_trials_norm"][::-1]]),
        y=pd.concat([
            df_opt["mean"] - df_opt["sem"],
            (df_opt["mean"] + df_opt["sem"])[::-1]
        ]),
        fill='toself',
        fillcolor=line_color.replace(')', ',0.1)').replace('rgb', 'rgba'),
        line=dict(color='rgba(255,255,255,0)'),
        hoverinfo="skip",
        showlegend=False
    ))


fig.update_layout(
    height=400,
    width=1100,
    template="plotly_white",
    font=dict(size=16),
    legend=dict(title="Optimizers", font=dict(size=16)),
    margin=dict(t=60, l=50, r=50, b=50)
)

fig.update_xaxes(title_text="Normalized Trials")
fig.update_yaxes(title_text="Difference in Performance HPI and Random")
fig.show()

/tmp/ipykernel_38293/129696559.py:29: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



,task_id,seed,n_trials_norm,diff_trial_value__cost_inc_norm
0,multi-objective/50/dev/yahpo/mo/iaml_glmnet/14...,0,0.00,0.000000
1,multi-objective/50/dev/yahpo/mo/iaml_glmnet/14...,0,0.02,0.000000
2,multi-objective/50/dev/yahpo/mo/iaml_glmnet/14...,0,0.04,0.000000
3,multi-objective/50/dev/yahpo/mo/iaml_glmnet/14...,0,0.06,0.000000
4,multi-objective/50/dev/yahpo/mo/iaml_glmnet/14...,0,0.08,0.000000
...,...,...,...,...
6013,multi-objective/50/dev/hpobench/multiobjective...,4,0.92,-0.005848
6014,multi-objective/50/dev/hpobench/multiobjective...,4,0.94,-0.005809
6015,multi-objective/50/dev/hpobench/multiobjective...,4,0.96,-0.005769
6016,multi-objective/50/dev/hpobench/multiobjective...,4,0.98,-0.005730
